In [211]:
import sys
import json
import pandas as pd
from collections import Counter

In [212]:
filename = "yelp_academic_dataset_business.json"

with open(filename, 'r', encoding='UTF-8') as f:
    data = f.readlines()
    # this line below may take at least 8-10 minutes of processing for 4-5 million rows. It converts all strings in list to actual json objects. 
    data = list(map(json.loads, data))
    
biz_df = pd.DataFrame(data)


In [213]:
biz_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC
3,211 W Monroe St,None,bFzdJJ3wp3PZssNEsyU23g,"Insurance, Financial Services",Phoenix,None,1,33.449999,-112.076979,Geico Insurance,,85003,8,1.5,AZ
4,2005 Alyth Place SE,{'BusinessAcceptsCreditCards': 'True'},8USyCYqpScwiNEb58Bt6CA,"Home & Garden, Nurseries & Gardening, Shopping...",Calgary,"{'Monday': '8:0-17:0', 'Tuesday': '8:0-17:0', ...",1,51.035591,-114.027366,Action Engine,,T2H 0N5,4,2.0,AB


In [214]:
biz_df.shape

(188593, 15)

In [215]:
biz_df["categories"][0:10]

0    Tours, Breweries, Pizza, Restaurants, Food, Ho...
1    Chicken Wings, Burgers, Caterers, Street Vendo...
2    Breakfast & Brunch, Restaurants, French, Sandw...
3                        Insurance, Financial Services
4    Home & Garden, Nurseries & Gardening, Shopping...
5                                   Coffee & Tea, Food
6                                       Food, Bakeries
7                                    Restaurants, Thai
8                                 Mexican, Restaurants
9                Flowers & Gifts, Gift Shops, Shopping
Name: categories, dtype: object

In [216]:
categories_in_biz_df = list(biz_df["categories"])
len(categories_in_biz_df)  # Number of businesses on this data sheet

188593

In [217]:
"""
*** Add i to create list of 1's and 0's for the purpose of filtering
"""
qualified_res_count = 0
categories_restaurant = []
target = "Restaurants"
avoid = "Restaurant Supplies"
filter_list = []
for element in categories_in_biz_df:
    if type(element) == str:  # There are some cases the type of the value is not str 
        if target in element and avoid not in element:
            qualified_res_count += 1
            filter_list.append(1)
            temp_list = element.split(", ")
            for category in temp_list:
                categories_restaurant.append(category)
        else: 
            filter_list.append(0)
    else:
        filter_list.append(0)

In [218]:
qualified_res_count  # Number of restaurants on this data sheet

57169

In [219]:
biz_df['Restaurants'] = filter_list
biz_df_restaurants = biz_df.loc[biz_df['Restaurants'] == 1]
biz_df_restaurants.shape

(57169, 16)

In [220]:
len(categories_restaurant)

227053

In [221]:
len(set(categories_restaurant))  # Number of categories (uncleaned)

705

In [222]:
categories_restaurant_dict = dict(Counter(categories_restaurant))
categories_restaurant_counter_list = []
for key in categories_restaurant_dict:
    categories_restaurant_counter_list.append([key, categories_restaurant_dict[key]])
categories_restaurant_counter_list.sort(key=lambda x: x[1], reverse = True)
"""
As you can see below, when you scroll down to the bottom,
some weird categories pop up, such as "Hookah Bars" and "Beauty & Spas". 
Therefore, there is a need to set a threshold or use alternative way to clean.
"""
categories_restaurant_counter_list  # Categories and their occurence in descending order


[['Restaurants', 57169],
 ['Food', 11665],
 ['Nightlife', 7948],
 ['Bars', 7646],
 ['Sandwiches', 6912],
 ['Fast Food', 6812],
 ['American (Traditional)', 6659],
 ['Pizza', 6603],
 ['Burgers', 5126],
 ['Breakfast & Brunch', 5023],
 ['Italian', 4550],
 ['Mexican', 4419],
 ['Chinese', 4247],
 ['American (New)', 4230],
 ['Coffee & Tea', 3108],
 ['Cafes', 3056],
 ['Japanese', 2566],
 ['Chicken Wings', 2538],
 ['Seafood', 2369],
 ['Salad', 2350],
 ['Event Planning & Services', 2247],
 ['Sushi Bars', 2156],
 ['Delis', 1835],
 ['Canadian (New)', 1828],
 ['Asian Fusion', 1780],
 ['Mediterranean', 1744],
 ['Barbeque', 1716],
 ['Sports Bars', 1665],
 ['Specialty Food', 1587],
 ['Caterers', 1556],
 ['Steakhouses', 1531],
 ['Desserts', 1434],
 ['Bakeries', 1418],
 ['Indian', 1417],
 ['Thai', 1393],
 ['Pubs', 1382],
 ['Diners', 1351],
 ['Vietnamese', 1225],
 ['Middle Eastern', 1182],
 ['Greek', 1023],
 ['Vegetarian', 1017],
 ['French', 977],
 ['Wine Bars', 916],
 ['Cocktail Bars', 849],
 ['Ice Crea

In [243]:
"""
Create dummy variables for categories
Once we decide the ways to clean the category list shown in the last section, 
then we can create dummy variable with the codes below.
"""
trial_df = biz_df_restaurants
categories_in_biz_df_restaurants = list(biz_df_restaurants["categories"])
#categories_set = set(categories_restaurant)
categories_set = ["Food", "Bars"]
for element in categories_set:
    dummy = []
    for i in range(len(categories_in_biz_df_restaurants)):
        if element in str(categories_in_biz_df_restaurants[i]):
            dummy.append(1)
        else:
            dummy.append(0)
    trial_df[element] = dummy
    print(element, "---", Counter(dummy))  # Test whether the number of occurence is correct

/Users/xulian/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Food --- Counter({0: 39161, 1: 18008})
Bars --- Counter({0: 46576, 1: 10593})


In [244]:
trial_df.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state,Restaurants,Food,Bars
0,1314 44 Avenue NE,"{'BikeParking': 'False', 'BusinessAcceptsCredi...",Apn5Q_b6Nz61Tq4XzPdf9A,"Tours, Breweries, Pizza, Restaurants, Food, Ho...",Calgary,"{'Monday': '8:30-17:0', 'Tuesday': '11:0-21:0'...",1,51.091813,-114.031675,Minhas Micro Brewery,,T2E 6L6,24,4.0,AB,1,1,0
1,,"{'Alcohol': 'none', 'BikeParking': 'False', 'B...",AjEbIBw6ZFfln7ePHha9PA,"Chicken Wings, Burgers, Caterers, Street Vendo...",Henderson,"{'Friday': '17:0-23:0', 'Saturday': '17:0-23:0...",0,35.960734,-114.939821,CK'S BBQ & Catering,,89002,3,4.5,NV,1,1,0
2,1335 rue Beaubien E,"{'Alcohol': 'beer_and_wine', 'Ambience': '{'ro...",O8S5hYJ1SMc8fA4QBtVujA,"Breakfast & Brunch, Restaurants, French, Sandw...",Montréal,"{'Monday': '10:0-22:0', 'Tuesday': '10:0-22:0'...",0,45.540503,-73.599300,La Bastringue,Rosemont-La Petite-Patrie,H2G 1K7,5,4.0,QC,1,0,0
7,"3417 Derry Road E, Unit 103","{'Alcohol': 'none', 'BusinessAcceptsCreditCard...",6OuOZAok8ikONMS_T3EzXg,"Restaurants, Thai",Mississauga,None,1,43.712946,-79.632763,Thai One On,Ridgewood,L4T 1A8,7,2.0,ON,1,0,0
8,1440 N. Dysart Ave,"{'Alcohol': 'none', 'Ambience': '{'romantic': ...",8-NRKkPY1UiFXW20WXKiXg,"Mexican, Restaurants",Avondale,"{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W...",1,33.448106,-112.341302,Filiberto's Mexican Food,,85323,40,2.5,AZ,1,0,0


In [122]:
#rest_df.to_csv('yelp_business_and_reviews', sep='\t', encoding='utf-8')